In [1]:
import copy
import pytorch_lightning as pl
import torch
import torchvision
from torch import nn
from lightly.loss import DINOLoss
from lightly.models.modules import DINOProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.dino_transform import DINOTransform
from lightly.utils.scheduler import cosine_schedule
import os
from lightly.data import LightlyDataset
from tqdm.auto import tqdm
import numpy as np

/home/akansh-i2sc/anaconda3/envs/hlcv-light/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys 
sys.path.insert(0, '/home/akansh-i2sc/Desktop/Study/HLCV/Why-Self-Supervision-in-Time/src/modules')
import seco_dataset_temporal as seco

In [3]:
class DINO(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        backbone = nn.Sequential(*list(resnet.children())[:-1])
        input_dim = 512
        # instead of a resnet you can also use a vision transformer backbone as in the
        # original paper (you might have to reduce the batch size in this case):
        # backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=False)
        # input_dim = backbone.embed_dim

        self.student_backbone = backbone
        self.student_head = DINOProjectionHead(
            input_dim, 512, 64, 2048, freeze_last_layer=1
        )
        self.teacher_backbone = copy.deepcopy(backbone)
        self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
        deactivate_requires_grad(self.teacher_backbone)
        deactivate_requires_grad(self.teacher_head)

        self.criterion = DINOLoss(output_dim=2048, warmup_teacher_temp_epochs=5)

    def forward(self, x):
        y = self.student_backbone(x).flatten(start_dim=1)
        z = self.student_head(y)
        return z

    def forward_teacher(self, x):
        y = self.teacher_backbone(x).flatten(start_dim=1)
        z = self.teacher_head(y)
        return z

    def training_step(self, batch, batch_idx):
        momentum = cosine_schedule(self.current_epoch, 10, 0.996, 1)
        update_momentum(self.student_backbone, self.teacher_backbone, m=momentum)
        update_momentum(self.student_head, self.teacher_head, m=momentum)
        # views = batch[0]
        views_1 = batch[0]
        views_2 = batch[1]
        # views = [view.to(self.device) for view in views]
        views_1 = [view.to(self.device) for view in views_1[0:4:2]]
        views_2 = [view.to(self.device) for view in views_2[0:4:2]]
        views = views_1
        views.extend(views_2)               # Teporal local+global views from 1 and 2
        global_views = views_1[:1]          # Teporal global view 1
        global_views.extend(views_2[:1])    # Teporal global view 2
        # global_views = views[:2]
        teacher_out = [self.forward_teacher(view) for view in global_views]
        student_out = [self.forward(view) for view in views]
        loss = self.criterion(teacher_out, student_out, epoch=self.current_epoch)
        return loss

    def on_after_backward(self):
        self.student_head.cancel_last_layer_gradients(current_epoch=self.current_epoch)

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=0.001)
        return optim


In [4]:
path2data = "/home/akansh-i2sc/Desktop/Study/HLCV/SeCo_dataset/seco_100k/seasonal_contrast_100k/"
transform = DINOTransform(cj_prob = 0, random_gray_scale = 0,gaussian_blur = (0,0,0), solarization_prob = 0)
seco_dataset = seco.SeasonalContrastTemporal(root=path2data, transform=transform)

In [5]:
dataloader = torch.utils.data.DataLoader(
    seco_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    num_workers=4,
)

In [6]:
model = DINO()

In [7]:
accelerator = "gpu" if torch.cuda.is_available() else "cpu"
print(accelerator)
trainer = pl.Trainer(max_epochs=100, devices=1, accelerator=accelerator)

# trainer = pl.Trainer(
#     max_epochs=1,
#     devices="auto",
#     accelerator="gpu",
#     strategy="ddp",
#     sync_batchnorm=True,
#     replace_sampler_ddp=True,  # or replace_sampler_ddp=True for PyTorch Lightning <2.0
# )

trainer.fit(model=model, train_dataloaders=dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


gpu


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type               | Params
--------------------------------------------------------
0 | student_backbone | Sequential         | 11.2 M
1 | student_head     | DINOProjectionHead | 691 K 
2 | teacher_backbone | Sequential         | 11.2 M
3 | teacher_head     | DINOProjectionHead | 691 K 
4 | criterion        | DINOLoss           | 0     
--------------------------------------------------------
11.9 M    Trainable params
11.9 M    Non-trainable params
23.7 M    Total params
94.942    Total estimated model params size (MB)


Epoch 11:   0%|          | 1/312 [00:02<11:31,  2.22s/it, loss=3.59, v_num=2]  

...

Epoch 99:   0%|          | 1/312 [00:01<09:24,  1.81s/it, loss=1.91, v_num=2]  

/home/akansh-i2sc/anaconda3/envs/hlcv-light/lib/python3.11/site-packages/lightly/utils/scheduler.py:32: RuntimeWarning: Current step number 99 exceeds max_steps 10.
  warnings.warn(


Epoch 99: 100%|██████████| 312/312 [01:30<00:00,  3.43it/s, loss=1.61, v_num=2]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 312/312 [01:31<00:00,  3.43it/s, loss=1.61, v_num=2]


In [8]:
pretrained_resnet_backbone = model.teacher_backbone
state_dict = {"resnet18_parameters": pretrained_resnet_backbone.state_dict()}
torch.save(state_dict, "/home/akansh-i2sc/Desktop/Study/HLCV/Why-Self-Supervision-in-Time/src/models/pre-trained_weights/resnet18_model_dino_temporal_20k_100e.pth")